In [ ]:
dotnet build ../examples/userstorygenerator/UserStoryGenerator
dotnet build ../examples/batcheval/library

In [ ]:
#i "nuget:https://api.nuget.org/v3/index.json" 
#r "nuget:Microsoft.SemanticKernel, 1.0.1"
#r "nuget:OpenTelemetry.Exporter.Console, 1.7.0"
#r "../examples/userstorygenerator/UserStoryGenerator/bin/Debug/net8.0/UserStoryGenerator.dll"
#r "../examples/batcheval/library/bin/Debug/net8.0/BatchEvalCore.dll"

using Microsoft.DotNet.Interactive;
using Microsoft.SemanticKernel;
using UserStoryGenerator;
using BatchEval.Core;

In [ ]:
var openAiEndpoint = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Enter your OpenAI Endpoint: ");
var openAiKey = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Enter your OpenAI Key: ");
var deploymentName = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Enter your Deployment Name: ");

In [ ]:
var builder = Microsoft.SemanticKernel.Kernel.CreateBuilder();

builder.AddAzureOpenAIChatCompletion(
        deploymentName,
        openAiEndpoint,
        openAiKey);

var kernel = builder.Build();

In [ ]:
internal struct UserInput
{
    public string Description { get; set; }
    public string ProjectContext { get; set; }
    public string Persona { get; set; }
}

In [ ]:
var dataFilePath = "assets/data.jsonl";
var preview = await BatchEval.Core.JsonlUtils.Preview<UserInput>(dataFilePath);
preview

In [ ]:
internal class UserStoryCreator : BatchEval.Core.IInputProcessor<UserInput>
{
    private readonly UserStorySkill userStoryGenerator;

    public UserStoryCreator(Microsoft.SemanticKernel.Kernel kernel)
    {
        this.userStoryGenerator = UserStorySkill.Create(kernel);
    }

    public async Task<ModelOutput> Process(UserInput userInput)
    {
        var userStory = await userStoryGenerator.GetUserStory(
            userInput.Description,
            userInput.ProjectContext,
            userInput.Persona);

        return new ModelOutput() {
            Input = $"Generate a user story for {userInput.Persona} so it can {userInput.Description}",
            Output = $"{userStory!.Title} - {userStory!.Description}"
        };
    }
}

In [ ]:
//var kernelFunctions = kernel.CreatePluginFromPromptDirectory("Prompts");

var batchEval = new BatchEval<UserInput>();

batchEval
    //.AddEvaluator(new PromptScoreEval("coherence", kernel, kernelFunctions["coherence"]))
    .AddEvaluator(new LenghtEval());

await batchEval
    .WithInputProcessor(new UserStoryCreator(kernel))
    .WithJsonl(dataFilePath)
    .Run();